### Challenge #13:  ECB cut-and-paste

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">
    
Write a k=v parsing routine, as if for a structured cookie. The routine should take:

```foo=bar&baz=qux&zap=zazzle```

... and produce:
    
```python
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```

</div>

In [41]:
def parse_structured_cookie(data):
    """
    For Challenge #13 (Set #2)
    I parse the supplied data and return as a Python dictionary
    """
    new_dict = {}
    split_data = data.split('&')

    for data_def in split_data:

        left_right = data_def.split('=')
        new_dict[left_right[0]] = left_right[1]

    return(new_dict)

In [42]:
parse_structured_cookie('foo=bar&baz=qux&zap=zazzle')

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}

<div class="alert alert-block alert-info">
    
Now write a function that encodes a user profile in that format, given an email address. You should have something like:
```python
profile_for("foo@bar.com")
```
<br>
... and it should produce:
<br><br> 
    
```python
{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
```
<br>
... encoded as:
<br><br>
    
```python
email=foo@bar.com&uid=10&role=user
```

</div>

In [43]:
def profile_for(email_address):
    """
    For Challenge #13 (Set #2)
    """
    
    email_address = email_address.replace('=', '')
    email_address = email_address.replace('&', '')
    return('email=' + email_address + '&uid=10&role=user')

In [44]:
profile_for("foo@bar.com")

'email=foo@bar.com&uid=10&role=user'

<div class="alert alert-block alert-info">
    
Your "profile_for" function should not allow encoding metacharacters (& and =). 

Eat them, quote them, whatever you want to do, but don't let people set their
email address to ```foo@bar.com&role=admin```.
    
</div>

In [45]:
profile_for("foo@bar.com&role=admin")

'email=foo@bar.comroleadmin&uid=10&role=user'

<div class="alert alert-block alert-info">
    
Now, two more easy functions. Generate a random AES key, then:

- Encrypt the encoded user profile under the key; "provide" that to the "attacker".
- Decrypt the encoded user profile and parse it.
    
</div>

In [46]:
# We'll want a PKCS#7 stripping function for this...
def strip_PKCS7_pad(data):
    """Function to remove PKCS#7 padding from a string"""
    if data[-data[-1]:].count(data[-1]) == data[-1]:
        return(data[:-data[-1]])
    else:
        return(-1)

In [47]:
user_profile = profile_for('someuser@somewhere.com')

unknown_key = bytes(list(randint(0, 256, 16)))
encrypted_profile = cp.AESEncrypt(user_profile, unknown_key)
decrypted_profile = strip_PKCS7_pad(cp.AESDecrypt(encrypted_profile, unknown_key))

parsed_profile = parse_structured_cookie(decrypted_profile.decode())
print(parsed_profile)

<div class="alert alert-block alert-info">
    
Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.

</div>
    
### How do we do this?

A couple of observations:

- We can supply an email, and get back a valid encrypted profile.  
- We know the block size (we learned how to figure this out earlier)
- We know the format of the unencrypted profile

First, lets construct a profile where the 'role=' portion stops at the end of an AES block:

In [55]:
# Make one account that has 'role=' stop precisely at end of 2nd AES block
email1 = 'aaaaaaa@a.com'
encrypted_profile1 = cp.AESEncrypt(profile_for(email1), unknown_key)

Remember, we don't actually know the key -- only the server knows that.  We're just doing it this way to simplify the coding a bit.

Now we have a plaintext that looks like this:

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|e  |m  |a  |i  |l  |=  | a | a | a | a | a | a | a | @ | a | . |   

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| c | o | m | & | u | i | d | = | 1 | 0 | & | r | o | l | e | = |

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| u | s | e | r |0xc|0xc|0xc|0xc|0xc|0xc|0xc|0xc|0xc|0xc|0xc|0xc|

Can you guess what's next?

Now make a second one where ```admin``` is start of 3rd block...padded out to end so there's nothing else.  Because this is using ECB, this is exactly how the 3rd AES block would look for a legitimate encrypted profile.

In [101]:
email2 = 'aaaaaaaaaaaaaaaaaaaaaaaaaaadmin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'
encrypted_profile2 = cp.AESEncrypt(profile_for(email2), unknown_key)

For this second profile, the plaintext looks like this:

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|e  |m  |a  |i  |l  |=  | a | a | a | a | a | a | a | a | a | a |

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | a | a | a | a | a | a | a | a | a | a | a | a | a | a | a |

|015|014|013|012|011|010|009|008|007|006|005|004|003|002|001|000|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| a | d | m | i | n |0xb|0xb|0xb|0xb|0xb|0xb|0xb|0xb|0xb|0xb|0xb|

Now the easy part...

We can cut & paste together full blocks of ciphertext from each profile to create the one we want... combine the parts of the encrypted data we need from each to create one that looks like a legitimate account for the original user, but with ```role=admin```.

In [102]:
evil_encrypted_admin_profile = encrypted_profile1[0:32] + encrypted_profile2[32:48]

Check if it worked.  Decrypt the "evil" encrypted profile & parse it. This is what the server would do...

In [103]:
decrypted_admin_profile = cp.AESDecrypt(evil_encrypted_admin_profile, unknown_key)
parse_structured_cookie(strip_PKCS7_pad(decrypted_admin_profile).decode())

{'email': 'aaaaaaa@a.com', 'uid': '10', 'role': 'admin'}

---
**Bingo!**

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)